In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from src.eval import evaluate_on_data, save_residual_plots, save_top_errors

DATA_PATH = Path("data/raw/house_prices_train.csv")     
SPLITS_DIR = Path("data/splits")          
TARGET_COL = "SalePrice"                  
MODEL_PATH = Path("models/model.joblib")

REPORTS_DIR = Path("reports")
FIGURES_DIR = REPORTS_DIR / "figures"
TEST_MD = REPORTS_DIR / "test_metrics.md"

In [2]:
# Define project directories
PROJECT_ROOT = Path("..").resolve()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
SPLITS_DIR = PROJECT_ROOT / "data" / "splits"
REPORTS_DIR = PROJECT_ROOT / "reports"
FIGURES_DIR = REPORTS_DIR / "figures"

# Create directories if they don't exist
DATA_RAW.mkdir(parents=True, exist_ok=True)
SPLITS_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)